# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from scipy import stats
from google.colab import drive

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [2]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/Eksperimen_SML_AkasBagusS/datasetraw/online_shoppers_intention.csv'
data = pd.read_csv(file_path)


Mounted at /content/drive


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [3]:
print('--- Info Dataset ---')
print(data.info())
print('\n--- Statistika Deskriptif ---')
print(data.describe())
print('\n--- Cek Missing Values ---')
print(data.isnull().sum())

--- Info Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  T

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [11]:
# 1. Mengatasi Missing Values
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Pastikan semua tipe data sudah benar setelah imputasi
data_imputed = data_imputed.convert_dtypes()

# 2. Menghapus Data Duplikat
data_imputed.drop_duplicates(inplace=True)

# 3. Normalisasi atau Standarisasi Fitur
# Memastikan kolom numerik benar-benar numerik
numerical_features = data_imputed.select_dtypes(include=['number']).columns

# Cek jika ada kolom numerik yang tidak bisa di-scaling
for col in numerical_features:
    if data_imputed[col].dtype not in ['float64', 'int64']:
        print(f"Kolom {col} memiliki tipe data tidak valid: {data_imputed[col].dtype}")
        data_imputed[col] = pd.to_numeric(data_imputed[col], errors='coerce')
        print(f"Perbaikan tipe data pada kolom {col}")

# Hilangkan missing value yang muncul akibat coercion
data_imputed.dropna(subset=numerical_features, inplace=True)

# Normalisasi atau Standarisasi menggunakan StandardScaler
scaler = StandardScaler()
data_imputed[numerical_features] = scaler.fit_transform(data_imputed[numerical_features])

# 4. Deteksi dan Penanganan Outlier
# Menghitung z-score
z_scores = np.abs(stats.zscore(data_imputed[numerical_features]))

# Jika z_scores memiliki dimensi berbeda, diatasi dengan memeriksa terlebih dahulu
if z_scores.shape[1] == len(numerical_features):
    threshold = 3
    data_imputed = data_imputed[(z_scores < threshold).all(axis=1)]
else:
    print("Dimensi z_scores tidak sesuai, outlier tidak dideteksi.")

# 5. Encoding Data Kategorikal
categorical_features = data_imputed.select_dtypes(include=['object', 'string']).columns

# Pastikan encoding dilakukan dengan benar
for col in categorical_features:
    le = LabelEncoder()
    try:
        data_imputed[col] = le.fit_transform(data_imputed[col].astype(str))
    except Exception as e:
        print(f"Encoding gagal pada kolom {col}: {e}")

# 6. Binning (Pengelompokan Data) pada fitur 'Administrative_Duration'
# Pastikan kolom tersebut ada dan tidak kosong sebelum di-binning
if 'Administrative_Duration' in data_imputed.columns:
    if not data_imputed['Administrative_Duration'].isnull().all():
        # Gunakan duplicates='drop' untuk menghindari error
        try:
            data_imputed['Administrative_Duration_Bin'] = pd.qcut(
                data_imputed['Administrative_Duration'],
                q=4,
                labels=False,
                duplicates='drop'  # Menyelesaikan error akibat bin edges duplikat
            )
        except ValueError as e:
            print(f"Terjadi kesalahan saat binning: {e}")
    else:
        print("Kolom 'Administrative_Duration' kosong, tidak bisa di-binning.")
else:
    print("Kolom 'Administrative_Duration' tidak ditemukan.")

# === Hasil Akhir Preprocessing ===
print('\n--- Dataset setelah Preprocessing ---')
print(data_imputed.head())
print('\n--- Informasi Dataset setelah Preprocessing ---')
print(data_imputed.info())

# === Menyimpan Dataset Hasil Preprocessing ===
output_path = '/content/drive/My Drive/Colab Notebooks/Eksperimen_SML_AkasBagusS/datasetraw/online_shoppers_intention_preprocessed.csv'
data_imputed.to_csv(output_path, index=False)
print(f"\n✅ Dataset hasil preprocessing berhasil disimpan di: {output_path}")
# === Menampilkan info file hasil preprocessing ===
file_size = os.path.getsize(output_path)  # ukuran dalam bytes
print(f"📄 Ukuran file: {file_size / 1024:.2f} KB")  # konversi ke KB

# Membaca kembali 5 baris pertama untuk validasi isi file
df_check = pd.read_csv(output_path)
print('\n--- Preview 5 Baris Pertama dari File CSV yang Disimpan ---')
print(df_check.head())

print('\n--- Info Dataset dari File CSV yang Disimpan ---')
print(df_check.info())

Kolom Administrative memiliki tipe data tidak valid: Int64
Perbaikan tipe data pada kolom Administrative
Kolom Administrative_Duration memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom Administrative_Duration
Kolom Informational memiliki tipe data tidak valid: Int64
Perbaikan tipe data pada kolom Informational
Kolom Informational_Duration memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom Informational_Duration
Kolom ProductRelated memiliki tipe data tidak valid: Int64
Perbaikan tipe data pada kolom ProductRelated
Kolom ProductRelated_Duration memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom ProductRelated_Duration
Kolom BounceRates memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom BounceRates
Kolom ExitRates memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom ExitRates
Kolom PageValues memiliki tipe data tidak valid: Float64
Perbaikan tipe data pada kolom PageValues
Kolom SpecialDay 

<ipython-input-11-8a2759421230>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_imputed[col] = le.fit_transform(data_imputed[col].astype(str))
<ipython-input-11-8a2759421230>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_imputed[col] = le.fit_transform(data_imputed[col].astype(str))
<ipython-input-11-8a2759421230>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu


✅ Dataset hasil preprocessing berhasil disimpan di: /content/drive/My Drive/Colab Notebooks/Eksperimen_SML_AkasBagusS/datasetraw/online_shoppers_intention_preprocessed.csv
📄 Ukuran file: 2787.89 KB

--- Preview 5 Baris Pertama dari File CSV yang Disimpan ---
   Administrative  Administrative_Duration  Informational  \
0        0.498791                -0.167036       0.385143   
1        0.498791                 0.140033      -0.398824   
2        2.000157                 2.341163      -0.398824   
3       -0.702302                -0.460019      -0.398824   
4       -0.702302                -0.460019      -0.398824   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0               -0.196759        0.312935                 0.979419   
1               -0.246257       -0.449536                -0.208767   
2               -0.246257       -0.270131                -0.019180   
3               -0.246257       -0.651367                -0.619937   
4               -0.2462